# JSON file generator
In the next code cell, change the parameter you want to loop for and it will creat as many .json file as you wish in current directory.

### Define Defult parameters

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#--------------------------------------#  1.4 Define Dirictiry and Load Model *(MOST IMPORTANT!)*

epoch = -1  #select previous checkpoint (-1 = don't use checkpoint)

#--------------------------------------#  1.1 System Physical Parameters

dx,dy,dz = 50e-9,50e-9,20e-9  # discretization (m) for magnetic field calculation (50nm is said to be a little bit too large)

nx = 100          # size x    (magnetic cells)
ny = 100          # size y    (magnetic cells)

Ms = 140e3    # saturation magnetization (A/m)
B0 = 60e-3    # bias field (T)

geometry_type = 1 #Module Geometry ["1 # Flip magnets", "2 #Permanent magnets (training multiplier)", "3 #Permanent magnets"] 
Np = 4 #number of actual probes

Ms_CoPt = 723e3 # saturation magnetization of the nanomagnets (A/m)
r0, dr, dm, z_off = 15, 4, 2, 10  # starting pos, period, magnet size, z distance
rx, ry = int((nx-2*r0)/dr), int((ny-2*r0)/dr+1) # build in calculation
damping_width = 10 # the damping boundary, in unit: cell 
sig_start_top=sig_start_bot=15 # signal generation line position, in unit: cell 

dt = 10e-12             # timestep (s)
timesteps = 1500   # number of timesteps for wave propagation (nx*5 is the defult wave propergation time for nxny=50,dt=20e-12)

#--------------------------------------#  1.2 Training Parameters


optimizer_name = 'Adam'   #can be SGD,Adam,Adagrad,RMSprop,Adadelta ... (if changed, remember to change optimizer_params as well)
optimizer_params = {}     #{'momentum': 0.9,'weight_decay': 1e-4}  # extra parameter for the optimizer if needed, if not use '{}'.
loss_function = {'train_loss_fn':'balanced_reduction_loss','pixel_w_loss_fn':'balanced_reduction_loss_ratio'}
#apply_softmax = True      # `True` if the loss function only takes input in range [0,1]
loss_fn_params = {'train_loss_fn':{'exceed_ratio': 0.021},'pixel_w_loss_fn':{'exceed_ratio': 0.05}}
square_after_sum = True

learn_rate = 0.025          # Learning rate
epoch_max = 50             # number of training epoch

#---------------------# pixel weighting the output during training

# additional training parameter
pixel_weighting_step = 0.002 #!# # the average weighting change shared across the selected pixels. (set to 0 to disable the method)
pixel_weighting_cap=(0.5,2.0) # the maximun and minimum the weighting can be

#--------------------------------------#  1.3 Define Training Data

Bt = 3e-3     # excitation field amplitude (T)
time_duration = 50 # `number of time step for each information`
normalize_superposition = 'sqrt'  # can be None, 'normal' or 'sqrt'
modulate_method = 'FM' # 'direct','FM', 'AM'(don't support AM yet)



test_epochs=[0,24,30,36,42,49]
#validation_ratio = 1/3  # ratio of training data used for training the final regressor
probe_time_resolution = 1e-9 # in seconds
silent_pixel = 4



#--------------------------------------#  1.3 Define Training Data
dataset_path=''
Bt = 3e-3     # excitation field amplitude (T)
time_duration = 200 # `number of time step for each information`

# `how the frequency and amplitude changes`

A=3.5e9
B=3.6e9

# define your parameters
frequencies = [
               [A,0,B,0], 
               [A,B,0],
               [B,0,A,0],
               [B,A,0]
               ]

normalize_superposition = 'sqrt'  # can be None, 'normal' or 'sqrt'


desired_out_list=[0,1,2,3] # the desired output for each signal 

#---------------------#  Increase input signal size with separate noise

#--model training signals--
noise_std = Bt* 0.1
clone = 20 #1 #--#3
seed=42

train_ratio = 0.2  # ratio of training data used for training the final regressor
val_ratio = 0.3



#-----------------!!!------------------# or we define signal from datasets

dataset_path = ''
train_ratio = 0.4
val_ratio = 0.3
batch_size = 2
distribute_data_in_multi_epochs = 5 # None for deactivate it 
f_band_start=3.5e9
f_bandwidth=1e9

#--------------------------------------# parameters for the virtual probe method

detection_width = 5

#'equal_probes' or 'gradient_probes'
probe_reading_method_Train = 'equal_probes' 
probe_reading_method_Test =  'equal_probes'

weighting_intersect = (0.0,0.2) # left value for normal label signal, right value for label ignore signal
weighting_sigma     = (2,2)
dist_cut_off_train  = (2.5,6) 
dist_cut_off_test = (1.5,7)

merge_ignore_with_last_probe=True #include the second probe intensity with ignore label vertual probes

#--------------------------------------#  1.5 Take note of the Model Detail


comments_for_this_training_and_testing= """
# Overall comment: 

"""


### Define changing parameters and creat files
add as many loop as you like, remember to set each file a different name and basedir.

In [2]:
import pickle
param_names =  ["basedir", "dx", "dy", "dz","nx","ny","Ms","B0","geometry_type","Np","Ms_CoPt", "r0", "dr", "dm", "z_off", "rx", "ry", 
                "damping_width", "sig_start_top", "sig_start_bot","dt", "timesteps", 
                "optimizer_name", "optimizer_params", "loss_function", "loss_fn_params","square_after_sum","apply_softmax", "learn_rate", "epoch_max","pixel_weighting_step","pixel_weighting_cap",
                "Bt", "time_duration", "frequencies", "normalize_superposition","modulate_method", "desired_out_list", 
                "noise_std","clone","seed","batch_size","noise_std_test","clone_test","seed_test","test_epochs",
                "validation_ratio","probe_time_resolution","silent_pixel","dataset_path","train_ratio","val_ratio",
                "distribute_data_in_multi_epochs","f_band_start","f_bandwidth",
                "detection_width","probe_reading_method_Train","probe_reading_method_Test","weighting_intersect","weighting_sigma","dist_cut_off_train","dist_cut_off_test",
                "merge_ignore_with_last_probe","comments_for_this_training_and_testing"]

In [3]:
import os
if not os.path.isdir('json_files'):
    os.makedirs('json_files')

In [4]:
A=3.5e9
B=3.7e9


for d_nx in np.linspace(60,200,15):
    for d_ny in np.linspace(60,200,15):
            para_used_str=f"(nx={int(d_nx)},ny={int(d_ny)})"
            basedir = f"v6.52_{para_used_str}_"
                            #-------------##-------------##-------------#
            time_duration = 200 # for the pulse default value

            delta_t = 200

            delta_t=int(delta_t)


            if delta_t>0:
                frequencies = [
                            [A,(0,f'{delta_t}'),B,0],  
                            [A,(0,f'{delta_t}'),A,0],
                            [B,(0,f'{delta_t}'),A,0],
                            [B,(0,f'{delta_t}'),B,0]
                            ]
            elif delta_t==0:
                frequencies = [
                            [A,B,0],  
                            [A,A,0],
                            [B,A,0],
                            [B,B,0]
                            ]
            elif delta_t<0:
                frequencies = [
                            [(A,f'{int(time_duration+delta_t)}'),((A,B),f'{int(-delta_t)}'),(B,f'{int(time_duration+delta_t)}'),0],  
                            [(A,f'{int(2*time_duration+delta_t)}'),0],
                            [(B,f'{int(time_duration+delta_t)}'),((B,A),f'{int(-delta_t)}'),(A,f'{int(time_duration+delta_t)}'),0],
                            [(B,f'{int(2*time_duration+delta_t)}'),0]
                            ]
            else:
                print('error in delta_t')
            desired_out_list=[0,1,2,3] # the desired output for each signal 

            #-------------#

            nx= int(d_nx)
            ny= int(d_ny)
            Ms_CoPt = 723e3 # saturation magnetization of the nanomagnets (A/m)
            r0, dr, dm, z_off = 15, 4, 2, 10  # starting pos, period, magnet size, z distance
            rx, ry = int((nx-2*r0)/dr), int((ny-2*r0)/dr+1) # build in calculation

            
            #-------------##-------------##-------------#
            parameters = {key: value for key, value in locals().items() if key in param_names and value is not None}
            with open(f"json_files/para_{para_used_str}.pickle", 'wb') as f:
                pickle.dump(parameters, f)
            print(parameters)

{'dx': 5e-08, 'dy': 5e-08, 'dz': 2e-08, 'nx': 60, 'ny': 60, 'Ms': 140000.0, 'B0': 0.06, 'geometry_type': 1, 'Np': 4, 'Ms_CoPt': 723000.0, 'r0': 15, 'dr': 4, 'dm': 2, 'z_off': 10, 'rx': 7, 'ry': 8, 'damping_width': 10, 'sig_start_top': 15, 'sig_start_bot': 15, 'dt': 1e-11, 'timesteps': 1500, 'optimizer_name': 'Adam', 'optimizer_params': {}, 'loss_function': {'train_loss_fn': 'balanced_reduction_loss', 'pixel_w_loss_fn': 'balanced_reduction_loss_ratio'}, 'loss_fn_params': {'train_loss_fn': {'exceed_ratio': 0.021}, 'pixel_w_loss_fn': {'exceed_ratio': 0.05}}, 'square_after_sum': True, 'learn_rate': 0.025, 'epoch_max': 50, 'pixel_weighting_step': 0.002, 'pixel_weighting_cap': (0.5, 2.0), 'Bt': 0.003, 'time_duration': 200, 'normalize_superposition': 'sqrt', 'modulate_method': 'FM', 'test_epochs': [0, 24, 30, 36, 42, 49], 'probe_time_resolution': 1e-09, 'silent_pixel': 4, 'dataset_path': '', 'frequencies': [[3500000000.0, (0, '200'), 3700000000.0, 0], [3500000000.0, (0, '200'), 3500000000.0, 